In [1]:
import visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import datetime
print(datetime.datetime.now())

2018-11-22 14:47:23.016000


In [3]:
rm = visa.ResourceManager()
rm.list_resources()

('ASRL3::INSTR',
 'ASRL10::INSTR',
 'GPIB0::3::INSTR',
 'GPIB0::7::INSTR',
 'GPIB0::12::INSTR',
 'GPIB0::24::INSTR')

In [3]:
ac_6221_1_id = 3
ac_6221__2id = 12
dc_2400_id = 24
nanovolt_id = 7

In [5]:
ac_1 = rm.open_resource('GPIB0::{}::INSTR'.format(ac_6221_1_id))
ac_2 = rm.open_resource('GPIB0::{}::INSTR'.format(ac_6221__2id))
dc = Keithley2400("GPIB::{}".format(dc_2400_id))
volt = rm.open_resource('GPIB0::{}::INSTR'.format(nanovolt_id))

In [6]:
def trigger_ac(ac, offs=0.005, amp=1e-6, duration=0.005, wait_after_arm=2):
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(offs))
    ac.write("SOUR:WAVE:AMPL {}".format(amp))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(duration))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(wait_after_arm)
    ac.write("SOUR:WAVE:INIT")

In [6]:
def init_communication():
    print("check ac_1: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_1, offs=0.00005, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(20)
    print("check ac_2: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_2, offs=0.00005, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(10)
    dc.apply_current()# Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = 100e-6            # Sets the source current to 0.1 mA
    dc.enable_source() 
    print("enable dc current now, current:" + str(dc))
    print("check nano volt:")
    for i in range(10):
        print(volt.query("fetch?"))
        time.sleep(0.1)
    time.sleep(10)
    dc.shutdown()   
    print("dc current disabled")


In [37]:
init_communication()

check ac_1: offerset 50uA, amp1uA, duration 10s
check ac_2: offerset 50uA, amp1uA, duration 10s
enable dc current now, current:<pymeasure.instruments.keithley.keithley2400.Keithley2400 object at 0x0000000004BECAC8>
check nano volt:
+2.60861190E-07

+2.60861190E-07

-4.63351369E-05

-4.63351369E-05

-6.61458239E-05

-6.61458239E-05

-6.61462246E-05

-6.61462246E-05

-6.61488955E-05

-6.61488955E-05

dc current disabled


In [9]:
def measure_resistance(dc=dc, volt=volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.1):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(30):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v

In [10]:
def do_measurement(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005, round=6, point_per_round=5, wait_after_trigger=15):
    file_name = "./_switch/data/{}ac_{}uA{}ms_wait{}s.csv".format(sample_id, int(ac_current * 1e6), int(duration * 1e3), wait_after_trigger)
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")

    for i in range(round):
        print("round: " + str(i+1))
        for current in ['ac_1', 'ac_2']:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)

                else:
                    trigger_ac(ac_2, offs=ac_current, duration=duration)
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(20)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""

In [11]:
import subprocess
def set_T(t=300):
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise

In [12]:

def set_Field(field=0):
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise

In [39]:
do_task = True

In [ ]:
if not do_task:
    raise
t_array = [270, 300]
i = [8.5*1e-3, 9*1e-3, 9.5*1e-3, 10*1e-3, 10.5*1e-3, 11*1e-3]
fields = [0]
wait_after_T = 15 # in minute
for t in t_array:
    print(datetime.datetime.now())
    print("set T to:" + str(t))
    print(datetime.datetime.now())
    print("start measurment:")
    set_T(t)
    #time.sleep(wait_after_T*60)
    for c in i:
        for field in fields:
            print("current pulse:" + str(c))
            #print("set field to:" + str(field))
            #set_Field(field)
            print(datetime.datetime.now())
            do_measurement(sample_id='XD308_2_11_22_5nm_Cr2O3_0001_pt_current_'+ str(t) +"K" + str(field) + "G" + 
                                     str(int(time.time())),
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=5, point_per_round=10, wait_after_trigger=30)
do_task = False
#set_T(300)

2018-11-26 00:44:17.868000
set T to:270
2018-11-26 00:44:17.868000
start measurment:
current pulse:0.0095
2018-11-26 00:44:17.868000
round: 1
current:ac_1
-6.477294807499999e-05
-6.477864514285714e-05
-6.477336339285713e-05
-6.476466735714286e-05
-6.476497937142858e-05
-6.477458472857143e-05
-6.476591188571429e-05
-6.476489521071429e-05
-6.476868704642857e-05
-6.476674057857143e-05
current:ac_2
-6.479854537499999e-05
-6.479852373928572e-05
-6.478892729285714e-05
-6.481405056428572e-05
-6.480627745357142e-05
-6.481170236071428e-05
-6.481161657142857e-05
-6.481485082857142e-05
-6.480368704642856e-05
-6.480992721071428e-05
round: 2
current:ac_1
-6.477687666428571e-05


PermissionError: [Errno 13] Permission denied: './_switch/data/XD308_2_11_22_5nm_Cr2O3_0001_pt_current_test_270K0G1543211057ac_9500uA1ms_wait30s.csv'

In [109]:
set_Field(70000)
set_T(320)
time.sleep(3*60)
set_T(280)
set_Field(0)
set_T(300)
time.sleep(10*60)
t='300_field_cool'
field=0
do_measurement(sample_id='11_11_10nm_CrO_0001_field_test__dependent_'+ str(t) +"K" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=5, point_per_round=10, wait_after_trigger=30)

70000.4609375
current field 70000.4609375
current T 320.0111
current T 280.0459
-0.0250031203032
current field -0.0250031203032
current T 300.0447
round: 1
current:ac_1
5.338501771785714e-05
5.338310227499998e-05
5.337323551428571e-05
5.341053359999999e-05
5.341465729642857e-05
5.343359417500001e-05
5.345099588214286e-05
5.3444118699999996e-05
5.344992277857143e-05
5.349771033214286e-05
current:ac_2
5.345018065714285e-05
5.346342954285715e-05
5.347556795357143e-05
5.3464633303571424e-05
5.3441842346428576e-05
5.345311485e-05
5.3439813725e-05
5.344497488928571e-05
5.342081872142857e-05
5.341662200357142e-05
round: 2
current:ac_1
5.3411622824999996e-05
5.340672966428571e-05
5.341009960714286e-05
5.3409492421428574e-05
5.343453876428572e-05
5.3439622682142855e-05
5.3425656835714275e-05
5.342824685357142e-05
5.3460142014285714e-05
5.3453963932142864e-05
current:ac_2
5.3459970885714277e-05
5.347818038928572e-05
5.348913376785714e-05
5.348220914285714e-05
5.348417898571429e-05
5.347534838214

PermissionError: [Errno 13] Permission denied: './_switch/data/11_11_10nm_CrO_0001_field_test__dependent_300_field_coolK0Gac_12000uA1ms_wait30s.csv'

In [39]:
do_task = True


In [ ]:
if not do_task:
    raise

T_SLEEP = 15
field=0
COOL_TO_T = 150

t='300K_back_from_360K_no_field'
print(t)
set_T(360)
print("reach 360K, wait 30min")
time.sleep(30*60)

set_T(300)
print("reach 300K, wait 30min")
time.sleep(T_SLEEP*60)


do_measurement(sample_id='XD308_2_5nm_CrO_0001_field_dependent_'+ str(t) +"K" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=10, wait_after_trigger=30)
t='300K_back_from_360K_wtih_field_14T'
print(t)
set_T(360)
print("reach 360K, raising field")
set_Field(140000)
print("reach 14T, lowering to 150K")
set_T(COOL_TO_T)
print("decreasing Field")
set_Field(0)
print("Field is 0, raise to 300K")
set_T(300)
print("reach 300K, wait 15min")
time.sleep(T_SLEEP*60)


do_measurement(sample_id='XD308_2_5nm_CrO_0001_field_dependent_'+ str(t) +"K" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=10, wait_after_trigger=30)

t='300K_back_from_360K_wtih_field_2T'
print(t)
set_T(360)
print("reach 360K, raising field")
set_Field(20000)
print("reach 2T, lowering to 150K")
set_T(COOL_TO_T)
print("decreasing Field")
set_Field(0)
print("Field is 0, raise to 300K")
set_T(300)
print("reach 300K, wait 15min")
time.sleep(T_SLEEP*60)

do_measurement(sample_id='XD308_2_5nm_CrO_0001_field_dependent_'+ str(t) +"K" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=10, wait_after_trigger=30)


do_task = False
    

In [ ]:
if not do_task:
    raise

T_SLEEP = 15
field=0
COOL_TO_T = 150

t='300K_back_from_360K_no_field'
print(t)
set_T(360)
print("reach 360K, wait 30min")
time.sleep(30*60)

set_T(300)
print("reach 300K, wait 30min")
time.sleep(T_SLEEP*60)


do_measurement(sample_id='XD308_2_5nm_CrO_0001_field_dependent_'+ str(t) +"K" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=10, wait_after_trigger=30)
t='300K_back_from_360K_wtih_field_14T'
print(t)
set_T(360)
print("reach 360K, raising field")
set_Field(140000)
print("reach 14T, lowering to 150K")
set_T(COOL_TO_T)
print("decreasing Field")
set_Field(0)
print("Field is 0, raise to 300K")
set_T(300)
print("reach 300K, wait 15min")
time.sleep(T_SLEEP*60)


do_measurement(sample_id='XD308_2_5nm_CrO_0001_field_dependent_'+ str(t) +"K" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=10, wait_after_trigger=30)

t='300K_back_from_360K_wtih_field_2T'
print(t)
set_T(360)
print("reach 360K, raising field")
set_Field(20000)
print("reach 2T, lowering to 150K")
set_T(COOL_TO_T)
print("decreasing Field")
set_Field(0)
print("Field is 0, raise to 300K")
set_T(300)
print("reach 300K, wait 15min")
time.sleep(T_SLEEP*60)

do_measurement(sample_id='XD308_2_5nm_CrO_0001_field_dependent_'+ str(t) +"K" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=10, wait_after_trigger=30)


do_task = False
    

In [16]:
def stable_measurement(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005,
                       round=6, point_per_round=1, wait_after_trigger=15, dc_measure_delay=60, dc_measurement_count=30):
    file_name = "./_switch/data/{}ac_{}uA{}ms_wait{}s.csv".format(sample_id, int(ac_current * 1e6), int(duration * 1e3), wait_after_trigger)
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")

    for i in range(round):
        print("round: " + str(i+1))
        for current in ['ac_1', 'ac_2']:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)

                else:
                    trigger_ac(ac_2, offs=ac_current, duration=duration)
                time.sleep(wait_after_trigger)
                for i in range(dc_measurement_count):
                    time.sleep(dc_measure_delay)
                    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                    with open(file_name, 'a') as f:
                        f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                    print(average_v)
                time.sleep(20)

In [18]:
stable_measurement(sample_id='30nm_CrO_0001_330K_stable_1min_delay_30_count', ac_current=c, dc_current=100e-6, duration=0.001,
                   round=10, point_per_round=1, wait_after_trigger=30, dc_measure_delay=60, dc_measurement_count=30)

round: 1
current:ac_1
9.431330707142858e-05
9.43381000142857e-05
9.433512416428573e-05
9.434283410357144e-05
9.433206410714284e-05
9.433824302142855e-05
9.43364345857143e-05
9.431444373928572e-05
9.432693993214288e-05
9.431336129285715e-05
9.430465472857142e-05
9.430196330714288e-05
9.429996161785713e-05
9.430293407857143e-05
9.43006512107143e-05
9.428696045714285e-05
9.430295150357143e-05
9.428607778214287e-05
9.428342891785712e-05
9.429738762142855e-05
9.429405230357143e-05
9.427626310357143e-05
9.427893440714288e-05
9.426475354285716e-05
9.426020255714289e-05
9.426062128571426e-05
9.426518625357144e-05
9.427324081071429e-05
9.426125397857145e-05
9.424162587142856e-05
current:ac_2
9.49565684857143e-05
9.493103777499998e-05
9.491215283571429e-05
9.487708946785713e-05
9.488089966428574e-05
9.48385205714286e-05
9.483650877500002e-05
9.484322112857143e-05
9.482787673928571e-05
9.481444497142855e-05
9.482042050714288e-05
9.483155735714285e-05
9.4829030425e-05
9.482499039642856e-05
9.48133

PermissionError: [Errno 13] Permission denied: './_switch/data/30nm_CrO_0001_330K_stable_1min_delay_30_countac_10000uA1ms_wait30s.csv'

In [6]:
inst = rm.open_resource('GPIB0::7::INSTR')

In [7]:
inst.write("initiate")

(10, <StatusCode.success: 0>)

In [14]:
keithley = Keithley2400("GPIB::{}".format(dc_2400_id))
inst = rm.open_resource('GPIB0::{}::INSTR'.format(nanovolt_id))

In [42]:
keithley.apply_current()# Sets up to source current
keithley.source_current_range = 10e-3   # Sets the source current range to 10 mA
keithley.compliance_voltage = 10        # Sets the compliance voltage to 10 V
keithley.source_current = 100e-6            # Sets the source current to 0 mA
keithley.enable_source() 
v = []
average_v = []
for i in range(30):
    if i < 10:
        continue
    time.sleep(0.1)
    v.append(abs(float(inst.query("fetch?"))))

# Sets the compliance voltage to 10 V
keithley.source_current = -100e-6            # Sets the source current to 0 mA
keithley.enable_source() 

for i in range(30):
    time.sleep(0.1)
    if i < 10:
        continue
    v.append(abs(float(inst.query("fetch?"))))
v = sorted(v)
v = v[5: -5]

with open("/Users/sishengyu/Desktop/abc_3.csv", 'a') as f:
    print(np.array(v).mean())
    f.write(str(np.array(v).mean()))
    f.write(",")
    f.write(str(np.array(v).std()))
    f.write(",")
    f.write(str(np.array(v).std()/np.array(v).mean()))
    f.write("\n")

0.0003667652223333333
